In [ ]:
# | hide

from IPython.display import Markdown

In [ ]:
# | hide

try:
    import google.colab
    in_colab = True
except:
    in_colab = False

In [ ]:
# | hide

if in_colab:
    display(Markdown("""
# If you see this message, you are running in Google Colab
Along with this interactive tutorial the content of this notebook is organized and formatted for documentation purpuoses. 

You can ignore the '# | hide', '# | notest' and '# | echo: false' comments, they are not important for the tutorial.
    """))


# If you see this message, you are running in Google Colab
Along with this interactive tutorial the content of this notebook is organized and formatted for documentation purpuoses. 

You can ignore the '# | hide', '# | notest' and '# | echo: false' comments, they are not important for the tutorial.
    

In [ ]:
# | notest
# | hide

# This step is necessary to be able to run FastKafka inside Google Colab

import nest_asyncio

In [ ]:
# | notest
# | hide

nest_asyncio.apply()

# FastKafka


<b>Effortless Kafka integration for your web services</b>


---

![PyPI](https://img.shields.io/pypi/v/fastkafka)
![PyPI - Downloads](https://img.shields.io/pypi/dm/fastkafka)
![PyPI - Python Version](https://img.shields.io/pypi/pyversions/fastkafka)

![GitHub Workflow Status](https://img.shields.io/github/actions/workflow/status/airtai/fastkafka/test.yaml)
![CodeQL](https://github.com/airtai/fastkafka//actions/workflows/codeql.yml/badge.svg)
![Dependency Review](https://github.com/airtai/fastkafka//actions/workflows/dependency-review.yml/badge.svg)

![GitHub](https://img.shields.io/github/license/airtai/fastkafka)

---


[FastKafka](https://fastkafka.airt.ai/) is a powerful and easy-to-use Python library for building asynchronous services that interact with Kafka topics. Built on top of [Pydantic](https://docs.pydantic.dev/), [AIOKafka](https://github.com/aio-libs/aiokafka) and [AsyncAPI](https://www.asyncapi.com/), FastKafka simplifies the process of writing producers and consumers for Kafka topics, handling all the parsing, networking, task scheduling and data generation automatically. With FastKafka, you can quickly prototype and develop high-performance Kafka-based services with minimal code, making it an ideal choice for developers looking to streamline their workflow and accelerate their projects.

---

#### ⭐⭐⭐ Stay in touch ⭐⭐⭐

Please show your support and stay in touch by:

- giving our [GitHub repository](https://github.com/airtai/fastkafka/) a star, and

- joining our [Discord server](https://discord.gg/CJWmYpyFbc).

Your support helps us to stay in touch with you and encourages us to continue developing and improving the library. Thank you for your support!

---

#### 🐝🐝🐝 We were busy lately 🐝🐝🐝

![Activity](https://repobeats.axiom.co/api/embed/21f36049093d5eb8e5fdad18c3c5d8df5428ca30.svg "Repobeats analytics image")

## Install

FastKafka works on macOS, Linux, and most Unix-style operating systems. You can install base version of `fastkafka` with `pip` as usual:

```sh
pip install fastkafka
```

To install fastkafka with testing features please use:
```sh
pip install fastkafka[test]
```

To install fastkafka with asyncapi docs please use:
```sh
pip install fastkafka[docs]
```

To install fastkafka with all the features please use:
```sh
pip install fastkafka[test,docs]
```

In [ ]:
# | hide
# | notest

# Install FastKafka with testing and documentation dependencies

! pip install fastkafka[test,docs]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Tutorial

You can start an interactive tutorial in Google Colab by clicking the button below:

<a href="https://colab.research.google.com/github/airtai/fastkafka/blob/main/nbs/index.ipynb" target=”_blank”>
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" />
</a>

## Writing server code

To demonstrate FastKafka simplicity of using `@produces` and `@consumes` decorators, we will focus on a simple app.

The app will consume jsons containig positive floats from one topic, log them and then produce incremented values to another topic.

### Messages

FastKafka uses [Pydantic](https://docs.pydantic.dev/) to parse input JSON-encoded data into Python objects, making it easy to work with structured data in your Kafka-based applications. Pydantic's [`BaseModel`](https://docs.pydantic.dev/usage/models/) class allows you to define messages using a declarative syntax, making it easy to specify the fields and types of your messages.

This example defines one `Data` mesage class. This Class will model the consumed and produced data in our app demo, it contains one `NonNegativeFloat` field `data` that will be logged and "processed" before being produced to another topic.

These message class will be used to parse and validate incoming data in Kafka consumers and producers.

In [ ]:
from pydantic import BaseModel, Field, NonNegativeFloat


class Data(BaseModel):
    data: NonNegativeFloat = Field(
        ..., example=0.5, description="Float data example"
    )

### Application

This example shows how to initialize a FastKafka application.

It starts by defining  a dictionary called `kafka_brokers`, which contains two entries: `"localhost"` and `"production"`, specifying local development and production Kafka brokers. Each entry specifies the URL, port, and other details of a Kafka broker. This dictionary is used for both generating the documentation and later to run the actual server against one of the given kafka broker.

Next, an object of the `FastKafka` class is initialized with the minimum set of arguments:

- `kafka_brokers`: a dictionary used for generation of documentation

We will also import and create a logger so that we can log the incoming data in our consuming function.

In [ ]:
from fastkafka import FastKafka
from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Demo Kafka app",
    kafka_brokers=kafka_brokers,
)

### Function decorators

FastKafka provides convenient function decorators `@kafka_app.consumes` and `@kafka_app.produces` to allow you to delegate the actual process of

- consuming and producing data to Kafka, and

- decoding and encoding JSON encode messages

from user defined functions to the framework. The FastKafka framework delegates these jobs to AIOKafka and Pydantic libraries.

These decorators make it easy to specify the processing logic for your Kafka consumers and producers, allowing you to focus on the core business logic of your application without worrying about the underlying Kafka integration.

This following example shows how to use the `@kafka_app.consumes` and `@kafka_app.produces` decorators in a FastKafka application:

- The `@kafka_app.consumes` decorator is applied to the `on_input_data` function, which specifies that this function should be called whenever a message is received on the "input_data" Kafka topic. The `on_input_data` function takes a single argument which is expected to be an instance of the `Data` message class. Specifying the type of the single argument is instructing the Pydantic to use `Data.parse_raw()` on the consumed message before passing it to the user defined function `on_input_data`.

- The `@produces` decorator is applied to the `to_output_data` function, which specifies that this function should produce a message to the "output_data" Kafka topic whenever it is called. The `to_output_data` function takes a single float argument `data`. It it increments the data returns it wrapped in a `Data` object. The framework will call the `Data.json().encode("utf-8")` function on the returned value and produce it to the specified topic.

In [ ]:
@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: Data):
    logger.info(f"Got data: {msg.data}")
    await to_output_data(msg.data)


@kafka_app.produces(topic="output_data")
async def to_output_data(data: float) -> Data:
    processed_data = Data(data=data+1.0)
    return processed_data

## Testing the service

The service can be tested using the `Tester` instances which internally starts InMemory implementation of Kafka broker.

The Tester will redirect your consumes and produces decorated functions to the InMemory Kafka broker so that you can quickly test your app without the need for a running Kafka broker and all its dependencies.

In [ ]:
from fastkafka.testing import Tester

msg = Data(
    data=0.1,
)

# Start Tester app and create InMemory Kafka broker for testing
async with Tester(kafka_app) as tester:
    # Send Data message to input_data topic
    await tester.to_input_data(msg)

    # Assert that the kafka_app responded with incremented data in output_data topic
    await tester.awaited_mocks.on_output_data.assert_awaited_with(
        Data(data=1.1), timeout=2
    )

23-05-31 09:05:47.409 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._start() called
23-05-31 09:05:47.410 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
23-05-31 09:05:47.410 [INFO] fastkafka._testing.in_memory_broker: InMemoryBroker starting
23-05-31 09:05:47.411 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-05-31 09:05:47.412 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-05-31 09:05:47.425 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-05-31 09:05:47.426 [INFO] fastkafka._testing.in_memory_broker: AIOKafkaProducer patched start() called()
23-05-31 09:05:47.426 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
23-05-31 09:05:47.427 [INF

### Recap

We have created a simple fastkafka application.
The app will consume the `Data` from the `input_data` topic, log it and produce the incremented data to `output_data` topic.

To test the app we have:

1. Created the app

2. Started our Tester class which mirrors the developed app topics for testing purposes

3. Sent Data message to `input_data` topic

4. Asserted and checked that the developed service has reacted to Data message 

## Running the service

The service can be started using builtin faskafka run CLI command. Before we can do that, we will concatenate the code snippets from above and save them in a file `"application.py"`

In [ ]:
# | echo: false

kafka_app_source = """
from pydantic import BaseModel, Field, NonNegativeFloat

from fastkafka import FastKafka
from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

class Data(BaseModel):
    data: NonNegativeFloat = Field(
        ..., example=0.5, description="Float data example"
    )

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Demo Kafka app",
    kafka_brokers=kafka_brokers,
)

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: Data):
    logger.info(f"Got data: {msg.data}")
    await to_output_data(msg.data)


@kafka_app.produces(topic="output_data")
async def to_output_data(data: float) -> Data:
    processed_data = Data(data=data+1.0)
    return processed_data
"""

with open("application.py", "w") as source:
    source.write(kafka_app_source)

Markdown(
    f"""
```python
# content of the "application.py" file
{kafka_app_source}
```
"""
)


```python
# content of the "application.py" file

from pydantic import BaseModel, Field, NonNegativeFloat

from fastkafka import FastKafka
from fastkafka._components.logger import get_logger

logger = get_logger(__name__)

class Data(BaseModel):
    data: NonNegativeFloat = Field(
        ..., example=0.5, description="Float data example"
    )

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Demo Kafka app",
    kafka_brokers=kafka_brokers,
)

@kafka_app.consumes(topic="input_data", auto_offset_reset="latest")
async def on_input_data(msg: Data):
    logger.info(f"Got data: {msg.data}")
    await to_output_data(msg.data)


@kafka_app.produces(topic="output_data")
async def to_output_data(data: float) -> Data:
    processed_data = Data(data=data+1.0)
    return processed_data

```


In [ ]:
# | notest
# | hide

# We need to start a local Apache Kafka broker so that our service can connect to it

# To do that, we need JRE and Kafka toolkit installed.
# We can do the installation by running "fastkafka testing install_deps"

! fastkafka testing install_deps

23-05-31 10:28:28.037 [INFO] fastkafka._components.test_dependencies: Java is already installed.
23-05-31 10:28:28.147 [INFO] fastkafka._components.test_dependencies: Kafka is installed.
23-05-31 10:28:28.148 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...


In [ ]:
# | notest
# | hide

# Start the local Apache Kafka broker

from fastkafka.testing import ApacheKafkaBroker

broker = ApacheKafkaBroker(apply_nest_asyncio=True)

broker.start()

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKaf

'127.0.0.1:9092'

To run the service, use the FastKafka CLI command and pass the module (in this case, the file where the app implementation is located) and the app simbol to the command.

```shell fastkafka run --num-workers=1 --kafka-broker localhost application:kafka_app```

After running the command, you should see the following output in your command line:

In [ ]:
# | notest
# | echo: false

# This cell will run your app indefinitely, you need to stop it manually

!fastkafka run --num-workers=2 --kafka-broker localhost application:kafka_app

[6206]: 23-05-31 10:28:36.705 [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to 'localhost:9092'
[6206]: 23-05-31 10:28:36.706 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[6204]: 23-05-31 10:28:36.708 [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to 'localhost:9092'
[6204]: 23-05-31 10:28:36.709 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
[6204]: 23-05-31 10:28:36.747 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
[6204]: 23-05-31 10:28:36.747 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'localhost:9092', 'auto_offset_reset': 'latest', 'max_poll_records': 100}
[6204]: 23-05-31 10:28:3

In [ ]:
# | notest
# | hide

# Stop the local broker to keep the runtime clean

broker.stop()

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.stop(): entering...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 5771...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 5771 was already terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 5401...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 5401 was already terminated.
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.stop(): exited.


## Documentation

The kafka app comes with builtin documentation generation using [AsyncApi HTML generator](https://www.asyncapi.com/tools/generator).

AsyncApi requires Node.js to be installed and we provide the following convenience command line for it:

```sh
fastkafka docs install_deps
```

In [ ]:
# | notest
# | hide

if in_colab:
    display(Markdown("""
# Update Node.js
When running in Colab, we need to update Node.js first:
    """))


# Update Node.js
When running in Colab, we need to update Node.js first:
    

In [ ]:
# | notest
# | hide

if in_colab:
    !npm install -g n
    !n lts


changed 1 package in 255ms
     copying : node/18.16.0
   installed : v18.16.0 (with npm 9.5.1)


In [ ]:
# | notest
# | echo: false

! fastkafka docs install_deps

23-05-31 10:29:56.092 [INFO] fastkafka._components.docs_dependencies: AsyncAPI generator installed


To generate the documentation programatically you just need to call the following command:

```sh
fastkafka docs generate application:kafka_app
```

In [ ]:
# | notest
# | echo: false

! fastkafka docs generate application:kafka_app

23-05-31 10:29:56.904 [INFO] fastkafka._components.asyncapi: New async specifications generated at: '/content/asyncapi/spec/asyncapi.yml'
23-05-31 10:30:11.353 [INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
23-05-31 10:30:11.353 [INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /content/asyncapi/docs.




This will generate the *asyncapi* folder in relative path where all your documentation will be saved. You can check out the content of it with:

```sh
ls -l asyncapi
```

In [ ]:
# | notest
# | echo: false

! ls -l asyncapi

total 8
drwxr-xr-x 4 root root 4096 May 31 10:20 docs
drwxr-xr-x 2 root root 4096 May 31 10:19 spec


In docs folder you will find the servable static html file of your documentation. This can also be served using our `fastkafka docs serve` CLI command (more on that in our guides).

In spec folder you will find a asyncapi.yml file containing the async API specification of your application. 

We can locally preview the generated documentation by running the following command:

```sh
fastkafka docs serve application:kafka_app
```

In [ ]:
# | notest
# | hide

if in_colab:
    port_proxy = google.colab.output.eval_js(f"google.colab.kernel.proxyPort({8000})")

    display(Markdown(f"""
# Documentation link when runnining in Colab
To acces your documentation when serving in Colab, you can use [this link]({port_proxy}) 
(after you have run the fastkafka docs serve application:kafka_app command)
"""))


# Documentation link when runnining in Colab
To acces your documentation when serving in Colab, you can use [this link](https://3ei8kr48kzu-496ff2e9c6d22116-8000-colab.googleusercontent.com/) 
(after you have run the fastkafka docs serve application:kafka_app command)


In [ ]:
# | notest
# | echo: false

# This cell will serve docs indefinitely, you need to stop it manually

!fastkafka docs serve application:kafka_app

23-05-31 10:30:13.049 [INFO] fastkafka._components.asyncapi: New async specifications generated at: '/content/asyncapi/spec/asyncapi.yml'
23-05-31 10:30:26.288 [INFO] fastkafka._components.asyncapi: Async docs generated at 'asyncapi/docs'
23-05-31 10:30:26.288 [INFO] fastkafka._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /content/asyncapi/docs.


Serving documentation on http://127.0.0.1:8000
127.0.0.1 - - [31/May/2023 10:30:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2023 10:30:28] "GET /css/global.min.css HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2023 10:30:28] "GET /css/asyncapi.min.css HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2023 10:30:28] "GET /js/asyncapi-ui.min.js HTTP/1.1" 200 -
Interupting serving of documentation and cleaning up...


From the parameters passed to the application constructor, we get the documentation bellow:
```python
from fastkafka import FastKafka

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Demo Kafka app",
    kafka_brokers=kafka_brokers,
)
```

![Kafka_servers](https://raw.githubusercontent.com/airtai/fastkafka/main/nbs/images/screenshot-kafka-servers.png)

The following documentation snippet are for the consumer as specified in the code above:


![Kafka_consumer](https://raw.githubusercontent.com/airtai/fastkafka/main/nbs/images/screenshot-kafka-consumer.png)

The following documentation snippet are for the producer as specified in the code above:


![Kafka_producer](https://raw.githubusercontent.com/airtai/fastkafka/main/nbs/images/screenshot-kafka-producer.png)

Finally, all messages as defined as subclasses of *BaseModel* are documented as well: 


![Kafka_![Kafka_servers](https://raw.githubusercontent.com/airtai/fastkafka/main/nbs/images/screenshot-kafka-messages.png)](https://raw.githubusercontent.com/airtai/fastkafka/main/nbs/images/screenshot-kafka-messages.png)

## License

FastKafka is licensed under the Apache License 2.0

A permissive license whose main conditions require preservation of copyright and license notices. Contributors provide an express grant of patent rights. Licensed works, modifications, and larger works may be distributed under different terms and without source code.

The full text of the license can be found [here](https://raw.githubusercontent.com/airtai/fastkafka/main/LICENSE).